In [1]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [2]:
# TODO:
# ssh WRCO0 df -h / /opt/platform /etc/nova/instances
# Add new computes, add keys

In [3]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [4]:
platform=os.getenv('OS_PLATFORM', 'poc2')
display_platform(platform)
inventory = read_inventory("$HOME/env/{}_hosts.ini".format(platform))
conn = connectToCloud(platform)
#inventory

In [5]:
show_notebook_url(platform, host_ip="10.3.216.210", port=8888)

#### Servers status (ping servers in 'ping_check' group)

In [6]:
display_html_ping_all(inventory, False)

ping(hlm1[10.3.216.10]) ... ping(hlm1vm[10.3.219.30]) ... ping(hos1[10.3.219.31]) ... ping(hos2[10.3.219.32]) ... ping(hos3[10.3.219.33]) ... ping(hos_vip[10.3.216.30]) ... ping(wrcl1[10.3.219.131]) ... ping(wrcl2[10.3.219.132]) ... ping(wrcl_vip[10.3.219.130]) ... ping(wrco0[10.3.219.177]) ... ping(wrco1[10.3.219.178]) ... ping(wrco2[10.3.219.179]) ... ping(wrco3[10.3.219.134]) ... 

hlm1[10.3.216.10],OK: 0.140 msec
hlm1vm[10.3.219.30],OK: 0.405 msec
hos1[10.3.219.31],OK: 0.220 msec
hos2[10.3.219.32],OK: 0.199 msec
hos3[10.3.219.33],OK: 0.204 msec
hos_vip[10.3.216.30],OK: 0.230 msec
wrcl1[10.3.219.131],OK: 0.190 msec
wrcl2[10.3.219.132],OK: 0.177 msec
wrcl_vip[10.3.219.130],OK: 0.155 msec
wrco0[10.3.219.177],OK: 0.094 msec
wrco1[10.3.219.178],OK: 0.120 msec


#### Show VMs (servers)

In [7]:
showServerList( conn )

2 flavors
11 images
0 servers


name,status,flavor,image,addresses


#### Show server port status

In [8]:
display_html_ping_ports_all(inventory)

ping(hlm1[10.3.216.10:22]) ... ping(hlm1vm[10.3.219.30:22]) ... ping(hos1[10.3.219.31:22]) ... ping(hos2[10.3.219.32:22]) ... ping(hos3[10.3.219.33:22]) ... ping(hos_vip[10.3.216.30:22]) ... ping(wrcl1[10.3.219.131:22]) ... ping(wrcl2[10.3.219.132:22]) ... ping(wrcl_vip[10.3.219.130:22]) ... ping(wrco0[10.3.219.177:22]) ... ping(wrco1[10.3.219.178:22]) ... ping(wrco2[10.3.219.179:22]) ... ping(wrco3[10.3.219.134:22]) ... ping(wrco4[10.3.219.135:22]) ... ping(wrco5[10.3.219.135:22]) ... ping(wrco6[10.3.219.137:22]) ... ping(wrco7[10.3.219.138:22]) ... 

hlm1[10.3.216.10:22],OK
hlm1vm[10.3.219.30:22],OK
hos1[10.3.219.31:22],OK
hos2[10.3.219.32:22],OK
hos3[10.3.219.33:22],OK
hos_vip[10.3.216.30:22],OK
wrcl1[10.3.219.131:22],OK
wrcl2[10.3.219.132:22],OK
wrcl_vip[10.3.219.130:22],OK
wrco0[10.3.219.177:22],OK
wrco1[10.3.219.178:22],OK


#### Show Endpoint URLs status (check ports open)

In [9]:
display_html_endpoint_urls(conn)

10.3.216.130:8774 [compute],OK
10.3.216.130:9696 [network],OK
10.3.216.30:5000 [identity],OK
10.3.216.30:8776 [volume],OK
10.3.216.30:9292 [image],OK


In [10]:
#print(inventory)

import paramiko

#hosts = list(inventory['ssh_check'].keys()).sorted()
ssh_hosts = sorted(inventory['ssh_check'])

for host in ssh_hosts:    
    ip = inventory['hosts'][host]['ansible_host']
    user = inventory['hosts'][host]['ansible_user']
    pkey = inventory['hosts'][host]['ssh_key']
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, key_filename=pkey, look_for_keys=False)
    stdin, stdout, stderr = ssh.exec_command("uptime")
    
    line=str(stdout.read())
    #print(line)
    up_pos = line.find(" up ")
    if up_pos != -1:
        up_str = line[up_pos + 4:]
        comma_pos = up_str.find(",")
        if comma_pos != -1:
            comma_pos2 = up_str[comma_pos+1:].find(",")
            if comma_pos2 != -1:
                up_str=up_str[:comma_pos+comma_pos2+1]
                print(host + ":" + up_str)
    
#inventory

hlm1:100 days,  2:13
hlm1vm:98 days, 17:33
hos1:85 days, 20:37
hos2:85 days, 20:37
hos3:29 days, 22:49
hos_vip:85 days, 20:37
wrcl1:48 days, 21:11
wrcl2:58 days,  3:12
wrcl_vip:58 days,  3:12
wrco0:58 days, 38 min
wrco1:47 days, 20:41
wrco2:47 days, 23:56
wrco3:47 days, 23:52
wrco4:13 days, 20:48
wrco5:13 days, 20:48
wrco6:11 days, 16:28
wrco7:11 days, 16:27


In [11]:

! [ ! -d history ] && mkdir history

df_hosts = sorted(inventory['df_check'])

import datetime
import time

d = datetime.date.today().strftime("%B %d, %Y")
dt = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")

for host in df_hosts:    
    ip = inventory['hosts'][host]['ansible_host']
    user = inventory['hosts'][host]['ansible_user']
    pkey = inventory['hosts'][host]['ssh_key']
    df_check = inventory['hosts'][host]['df_check']
    
    #print(df_check)
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, key_filename=pkey, look_for_keys=False)
    #stdin, stdout, stderr = ssh.exec_command("df -n+2 " + df_check.replace(",", " "))
    
    history_file='history/df_history_' + platform + '_' + host + '.txt'
    history_fd = open(history_file, 'a')
    
    full_df_cmd="df 2>&1"
    stdin, stdout, stderr = ssh.exec_command(full_df_cmd)
    #time.sleep(1)
    df_op = stdout.read().decode('utf8')
    history_fd.write(dt + '\n' + df_op)
    
    df_cmd="df " + df_check.replace(",", " ") + "| grep -v ^Filesystem"
    stdin, stdout, stderr = ssh.exec_command(df_cmd)
    df_op = stdout.read().decode('utf8')
    #print("HOST[" + host + "]<" + df_check + ">{" + df_cmd +"}:" + df_op)
    
    df_lines=df_op.split("\n")
    for df_line in df_lines:
        #print("LINE: " + df_line)
        pc_pos = df_line.find("%")
        if pc_pos != -1:
            pc=int(df_line[pc_pos-3:pc_pos])
            partn=df_line[pc_pos+1:]
            print(host + " " + str(pc) + "% " + partn)


hlm1 21%  /
hlm1vm 4%  /
hos1 50%  /
hos2 51%  /
hos3 52%  /
wrcl1 23%  /
wrcl1 1%  /scratch
wrcl1 11%  /var/log
wrcl1 1%  /opt/backups
wrcl1 23%  /
wrcl1 23%  /
wrcl1 23%  /
wrcl1 23%  /
wrcl2 22%  /
wrcl2 4%  /scratch
wrcl2 11%  /var/log
wrcl2 1%  /opt/backups
wrcl2 2%  /var/lib/rabbitmq
wrcl2 4%  /var/lib/postgresql
wrcl2 25%  /opt/cgcs
wrcl2 1%  /opt/platform
wrco0 11%  /
wrco0 2%  /var/log
wrco0 1%  /scratch
wrco0 1%  /local_pv
wrco0 25%  /etc/nova/instances
wrco0 1%  /opt/platform
wrco1 11%  /
wrco1 3%  /var/log
wrco1 1%  /scratch
wrco1 1%  /local_pv
wrco1 25%  /etc/nova/instances
wrco1 1%  /opt/platform
wrco2 11%  /
wrco2 5%  /var/log
wrco2 1%  /scratch
wrco2 1%  /local_pv
wrco2 25%  /etc/nova/instances
wrco2 1%  /opt/platform
wrco3 11%  /
wrco3 3%  /var/log
wrco3 1%  /scratch
wrco3 1%  /local_pv
wrco3 25%  /etc/nova/instances
wrco3 1%  /opt/platform
wrco4 11%  /
wrco4 3%  /var/log
wrco4 1%  /scratch
wrco4 1%  /local_pv
wrco4 25%  /etc/nova/instances
wrco4 1%  /opt/platform
wrco

In [12]:
# TODO: 
# equivalent of openstack service list

# See: how service list is implemented:
#    https://github.com/openstack/python-openstackclient/blob/master/openstackclient/compute/v2/service.py
# (compute_client = client_manager.compute)
#
# See also: how to create/use a clientmanager:
#    https://github.com/openstack/python-openstackclient/blob/master/examples/osc-lib.py

#from openstackclient.common import clientmanager
#from openstackclient.common import utils

